# Zero to Hero

## 1.12 Funcion de Ganancia de Arbol training/testing

El objetivo es poner el notebook anterior dentro de una funcion, e invocarla con distintas semillas, para maravillarnos ante la dispersion de las ganancias

In [1]:
rm( list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")
library("caret")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("~/dataScience/maestriaDC/2021/DM_en_EyF_2021cuat2/")  #se establece el directorio de trabajo

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,568028,30.4,1230412,65.8,734608,39.3
Vcells,1066160,8.2,8388608,64.0,1820723,13.9


Loading required package: lattice

Loading required package: ggplot2



In [2]:
GananciaArbol  <-  function( semilla, data, x, train=0.70) {
  #establezco la semilla
  set.seed(semilla)
  train_rows <- createDataPartition(dataset$clase_ternaria, p= 0.70,list= FALSE)
  
  modelo <-  rpart::rpart(formula= "clase_ternaria ~ ." ,
                          data= data[ train_rows],  #los datos de training
                          xval= 0,
                          control= x)
    
  #Aplico el modelo a los datos de testing  
  prediccion  <- predict( modelo, data[ -train_rows], type = "prob")
  
  prob_baja2  <- prediccion[, "BAJA+2"]
  ganancia_testing <- data[ -train_rows, sum(  (prob_baja2>0.025) * ifelse( clase_ternaria=="BAJA+2", 48750, -1250) )]
  
  return( ganancia_testing)
}

Vale la pena notar que dentro de la funcion no se han creado  dtrain y dtest,  sino que directamente se utilizan:
* data[ train_rows]  como training
* data[ -train_rows]  como testing


#Aqui empieza el programa

In [3]:
dataset <- data.table::fread("./datasetsOri/paquete_premium_202009.csv")   #cargo el dataset

In [4]:
#defino unos buenos hiperparametros
param  <- list("cp"= -0.5,
               "minsplit"=  900,
               "minbucket"= 440,
               "maxdepth"= 5
              )

In [ ]:
#Ahora hago algunas llamadas a la funcion

In [5]:
GananciaArbol(  11, dataset, x=param)

[1] 625000

In [6]:
GananciaArbol(  13, dataset, x=param)

[1] 886250

In [7]:
GananciaArbol(  17, dataset, x=param)

[1] 1696250

In [8]:
GananciaArbol(  19, dataset, x=param)

[1] 637500

In [9]:
GananciaArbol(  23, dataset, x=param)

[1] 791250

Lo primero que se obseva es la gran variabilidad de la ganancia segun la semilla, hasta un 50%

| semilla | ganancia |
| --- | ---: |
| 11 | 2093750 |
| 13 | 2142500 |
| 17 | 3070000 |
| 19 | 2576250 |
| 23 | 2280000 |



Esta dispersión es **NOTABLE**  ya que el algoritmo es el mismo, con los mismos parámetros.
<br>
Lo único que cambia es que datos se utilizan para entrenar y testear, pero siempre es 70% / 30%
<br>
Es más notable aún que se ha tenido cuidado que la partición sea estratificada segun el campo **clase_ternaria**, lo que apriori uno supondria que va a generar particiones muy homogéneas.